<a href="https://colab.research.google.com/github/rdwn354/E-Blink/blob/master/colab/interfrance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import tensorflow as tf

from keras.preprocessing.image import load_img, img_to_array
from time import time
from matplotlib import pyplot as plt
import os

In [ ]:
# path dataset
open = "/content/gdrive/MyDrive/datasets/dataset_modified/iman/dataset_iman_21.jpg"
close = "/content/gdrive/MyDrive/datasets/dataset_modified/iman/dataset_iman_2.jpg"

In [ ]:
image = load_img(close, target_size=(224,224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
print(image.shape)

(1, 224, 224, 3)


## PREDICTION USING H5


In [ ]:
keras_model_size = os.path.getsize('/content/gdrive/MyDrive/datasets/model/model_EBlink.h5')/1048576
print("keras model size : ", keras_model_size)

keras model size :  31.901939392089844


In [ ]:
model = tf.keras.models.load_model('/content/gdrive/MyDrive/datasets/model/model_EBlink.h5')

In [ ]:
time_before = time()
keras_prediction = model.predict(image)
time_after = time()
total_keras_time = time_after - time_before
print("total predifction time : ", total_keras_time)
print("prediction image : ", keras_prediction, "1 = close, 0 = open")

1/1 [==============================] - 2s 2s/step
total predifction time :  2.2072315216064453
prediction image :  [[1.]] 1 = close, 0 = open


## PREDICTION USING TFLITE WITOUT OPT


In [ ]:
tflite_path = os.path.getsize('/content/gdrive/MyDrive/datasets/model/model_EBlink.tflite')/1048576
print("tflite model size without opt : ", tflite_path)

tflite model size without opt :  10.621261596679688


In [ ]:
tflite_model_path = '/content/gdrive/MyDrive/datasets/model/model_EBlink.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']
print(input_shape)

[  1 224 224   3]


In [ ]:
input_data = image

In [ ]:
time_before=time()
interpreter.invoke()
time_after=time()
total_tflite_time = time_after - time_before
print("Total prediction time for tflite without opt model is: ", total_tflite_time)

output_data_tflite = interpreter.get_tensor(output_details[0]['index'])
print("The tflite w/o opt prediction for this image is: ", output_data_tflite)


Total prediction time for tflite without opt model is:  0.01718449592590332
The tflite w/o opt prediction for this image is:  [[2.415791e-15]]


## PREDICTION USING TFLITE WITH OPT

In [ ]:
tflite_optimized_size = os.path.getsize("/content/gdrive/MyDrive/datasets/model/model_EBlink_optimize.tflite")/1048576  #Convert to MB
print("tflite Model with optimization size is: ", tflite_optimized_size, "MB")
#Optimized using default optimization strategy (file size = 135MB). Taking about 39 seconds for prediction
tflite_optimized_model_path = "/content/gdrive/MyDrive/datasets/model/model_EBlink_optimize.tflite"


# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tflite_optimized_model_path)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on input data.
input_shape = input_details[0]['shape']
print(input_shape)

# Load image
input_data = image

interpreter.set_tensor(input_details[0]['index'], input_data)

time_before=time()
interpreter.invoke()
time_after=time()
total_tflite_opt_time = time_after - time_before
print("Total prediction time for tflite model with opt is: ", total_tflite_opt_time)

output_data_tflite_opt = interpreter.get_tensor(output_details[0]['index'])
print("The tflite with opt prediction for this image is: ", output_data_tflite_opt, " 0=Uninfected, 1=Parasited")


tflite Model with optimization size is:  2.6606826782226562 MB
[  1 224 224   3]
Total prediction time for tflite model with opt is:  0.023905277252197266
The tflite with opt prediction for this image is:  [[1.]]  0=Uninfected, 1=Parasited


In [ ]:
#Summary
print("###############################################")
print("Keras Model size is: ", keras_model_size)
print("tflite Model without opt. size is: ", tflite_path)
print("tflite Model with optimization size is: ", tflite_optimized_size)
print("________________________________________________")
print("Total prediction time for keras model is: ", total_keras_time)
print("Total prediction time for tflite without opt model is: ", total_tflite_time)
print("Total prediction time for tflite model with opt is: ", total_tflite_opt_time)
print("________________________________________________")
print("The keras prediction for this image is: ", keras_prediction)
print("The tflite w/o opt prediction for this image is: ", output_data_tflite)
print("The tflite with opt prediction for this image is: ", output_data_tflite_opt)

###############################################
Keras Model size is:  31.901939392089844
tflite Model without opt. size is:  10.621261596679688
tflite Model with optimization size is:  2.6606826782226562
________________________________________________
Total prediction time for keras model is:  2.2072315216064453
Total prediction time for tflite without opt model is:  0.01718449592590332
Total prediction time for tflite model with opt is:  0.023905277252197266
________________________________________________
The keras prediction for this image is:  [[1.]]
The tflite w/o opt prediction for this image is:  [[2.415791e-15]]
The tflite with opt prediction for this image is:  [[1.]]
